In [1]:
import openai
import pickle
import time

In [2]:
openai.api_key = "" # Use your own API Key here

In [17]:
chat_output_file = './chatgpt_responses/output.pickle'
chat_output_bm_25_file = './chatgpt_responses/output_bm25.pickle'

number_q_for_eval = 100

In [4]:
def read_pickle(input_file):
    with open(input_file, 'rb') as f:
       out = pickle.load(f)
    f.close()
    return out

def write_pickle(output, output_file):
    with open(output_file, 'wb') as f:
       pickle.dump(output, f, protocol=pickle.HIGHEST_PROTOCOL)
    f.close()

In [5]:
def read_bm25(): # these are actually the test files, before segregation into train val test
    with open("dev_v2.1_query_details_passages", "rb") as fp:   # Unpickling
        query_details_passages = pickle.load(fp)

    with open("passages.pickle", "rb") as fp:   # Unpickling
        passages = pickle.load(fp)

    with open("dev_okapi_bm_25_top10", "rb") as fp:   # Unpickling
        dev_okapi_bm_25_top10 = pickle.load(fp)
        
    return (query_details_passages, passages, dev_okapi_bm_25_top10)

def get_query_and_top_passages(queries, passages, bm25_top10, index):
    query = queries[index]
    passage_list = list(passages[i] for i in bm25_top10[index])
    return (query, passage_list)
    
(queries, passages, bm25_top10) = read_bm25()

In [6]:
"""
for psg in queries[0]['passages']:
    print(psg)
    print()
"""

"\nfor psg in queries[0]['passages']:\n    print(psg)\n    print()\n"

In [7]:
"""
(query, passage_list) = get_query_and_top_passages(queries, passages, bm25_top10, 2)
print(query['query'])
print()
print(query['wellFormedAnswer'])
print()
print(query['selectedPassage'])
print()
for p in passage_list[0:10]:
    print(p)
    print()
"""

"\n(query, passage_list) = get_query_and_top_passages(queries, passages, bm25_top10, 2)\nprint(query['query'])\nprint()\nprint(query['wellFormedAnswer'])\nprint()\nprint(query['selectedPassage'])\nprint()\nfor p in passage_list[0:10]:\n    print(p)\n    print()\n"

In [19]:
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages = [
     {"role": "system", "content" : "You’re very good at comprehending reading comprehesions and finding answers to questions from them."}
    ]
)
print(completion.choices[0].message.content)

Thank you, as an AI language model, it is what I was designed to do and I strive to provide accurate and helpful responses.


In [30]:
def chat_inference(chat_output_file, queries, passages, bm25_top10, number_queries=3, ground_truth=True):
    chat_responses = []
    for index in range(number_queries):
        (query, ret_passages) = get_query_and_top_passages(queries, passages, bm25_top10, index) 
        passage_list = query['passages']
        if (ground_truth == False):
            passage_list = ret_passages
        query_text = query['query']

        if len(passage_list) == 10:
            prompt = "Given a question and 10 passages, find a well formed answer sentence. Do not mention the source in the 'answer'. \n Q: {q} \n P1: {p1} \n P2: {p2} \n P3: {p3} \n P4: {p4} \n P5: {p5} \n P6: {p6} \n P7: {p7} \n P8: {p8} \n P9: {p9} \n P10: {p10} \n".format(q=query_text, p1=passage_list[0], p2=passage_list[1], p3=passage_list[2], p4=passage_list[3], p5=passage_list[4],
            p6=passage_list[5], p7=passage_list[6], p8=passage_list[7], p9=passage_list[8], p10=passage_list[9])
        elif len(passage_list) == 8:
            # there are few outlier queries with just 8 grouth truth passages
            prompt = "Given a question and 8 passages, find a well formed answer sentence. Do not mention the source in the 'answer'. \n Q: {q} \n P1: {p1} \n P2: {p2} \n P3: {p3} \n P4: {p4} \n P5: {p5} \n P6: {p6} \n P7: {p7} \n P8: {p8} \n".format(q=query_text, p1=passage_list[0], p2=passage_list[1], p3=passage_list[2], p4=passage_list[3], p5=passage_list[4],
            p6=passage_list[5], p7=passage_list[6], p8=passage_list[7])
        else:
            raise Exception("Passage list len not compatible!!")
        #print(prompt)

        input_dict = [{"role": "user", "content": prompt}]

        completion = openai.ChatCompletion.create(
          model="gpt-3.5-turbo-0301",
          messages=input_dict
        )

        chat_response = completion.choices[0].message.content
        #print(f'ChatGPT: {chat_response}')
        #print()
        chat_responses.append(chat_response)
        time.sleep(21) # imp to add this as ChatGPT has a rate limit of 3 request/min, so in every 20s
    print(len(chat_responses))
    assert len(chat_responses) == number_queries
    write_pickle(chat_responses, chat_output_file)

In [31]:
chat_inference(chat_output_bm_25_file, queries, passages, bm25_top10, number_q_for_eval, False)

100


In [32]:
y = read_pickle(chat_output_bm_25_file)

In [33]:
len(y)

100

In [34]:
y[0:10]

['The population of Albany, MN according to the 2000 Census is not specified in any of the given passages.',
 'There is no well formed answer sentence as none of the passages provide information about the current weather in Volcano, CA except for passage 7 which provides a detailed summary and forecast for the area.',
 'Hippocrates is considered the father of modern medicine.',
 'An appraisal for a Fannie loan is good for 120 days and must be current through the date of close, although there may be circumstances where the age of the appraisal may exceed this expiration under certain guidelines.',
 'The average salary for pharmacy technicians can vary greatly depending on factors such as location, industry, experience, and benefits. According to various sources, the average salary ranges between $28,000 to $32,000 per year.',
 'An average apple contains 58 calories.',
 'On average, a central air conditioning system can last for about 15 years, but the actual lifespan can vary depending 